In [1]:
import polars as pl
from db_functions import *
from config import *
from genai_functions import *

C:\Users\k_jal\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df1 = pl.read_parquet('jobs_from_7.0_days_2025-04-18.parquet').rename(
    {"Seniority level": "seniority_level",
     "Employment type": "employment_type",
     "Job function": "job_function",
     "Industries": "industries",}
)
df2 = pl.read_csv('joblist_2025-04-13_.csv').drop('cloud_focus')

In [3]:
# --- 1. Configuration ---
db_file = 'my_project.duckdb' # Use .duckdb extension by convention
table_name = 'test_table' #Choose a name for your SQL table


In [4]:
print(df1.count())

shape: (1, 10)
┌───────┬─────────┬──────────┬──────┬───┬───────────────┬──────────────┬──────────────┬────────────┐
│ title ┆ company ┆ location ┆ date ┆ … ┆ seniority_lev ┆ employment_t ┆ job_function ┆ industries │
│ ---   ┆ ---     ┆ ---      ┆ ---  ┆   ┆ el            ┆ ype          ┆ ---          ┆ ---        │
│ u32   ┆ u32     ┆ u32      ┆ u32  ┆   ┆ ---           ┆ ---          ┆ u32          ┆ u32        │
│       ┆         ┆          ┆      ┆   ┆ u32           ┆ u32          ┆              ┆            │
╞═══════╪═════════╪══════════╪══════╪═══╪═══════════════╪══════════════╪══════════════╪════════════╡
│ 31    ┆ 31      ┆ 31       ┆ 31   ┆ … ┆ 31            ┆ 31           ┆ 31           ┆ 31         │
└───────┴─────────┴──────────┴──────┴───┴───────────────┴──────────────┴──────────────┴────────────┘


In [14]:
con = duckdb.connect("my_project.duckdb")

In [6]:
con.sql("""SELECT COUNT(*) FROM test_table""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│           60 │
└──────────────┘

In [7]:
df = con.sql("""SELECT * FROM test_table
             LIMIT 5""")

In [8]:
df.show()

┌──────────────────────┬───────────────┬───────────────────────────────────────────────┬────────────┬────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [9]:
genai_list: list[dict] = []
for row in df1.head(5).rows(named=True):
    
    genai_data: dict = {}
    genai_data.update(classify_job_description(row['job_description']))
    genai_data.update({'job_url': row['job_url']})
    genai_list.append(genai_data)
    
print(genai_list)

Attempt 1/3: Calling Gemini API...
Successfully parsed JSON (Attempt 1).
Attempt 1/3: Calling Gemini API...
Successfully parsed JSON (Attempt 1).
Attempt 1/3: Calling Gemini API...
Successfully parsed JSON (Attempt 1).
Attempt 1/3: Calling Gemini API...
Successfully parsed JSON (Attempt 1).
Attempt 1/3: Calling Gemini API...
Successfully parsed JSON (Attempt 1).
[{'task_clarity': 'High', 'seniority_level_ai': 'Mid-Senior', 'requires_degree_it': 'No', 'mentions_certifications': 'No', 'years_of_experience': '3+', 'is_in_english': 'No', 'cloud_preference': 'GCP', 'skills_mentioned': ['Develop pipelines or ETL/ELT processes', 'Automation/Orchestration (Airflow, Prefect, Dagster, etc.)', 'Version control (GIT or similar)', 'Collaboration with data scientists or analysts'], 'job_url': 'https://www.linkedin.com/jobs/view/4191440590/'}, {'task_clarity': 'Medium', 'seniority_level_ai': 'Senior', 'requires_degree_it': 'No', 'mentions_certifications': 'No', 'years_of_experience': '3+', 'is_in_eng

In [10]:
df_genai = pl.DataFrame(genai_list)

In [11]:
print(df_genai)

shape: (5, 9)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ task_clar ┆ seniority ┆ requires_ ┆ mentions_ ┆ … ┆ is_in_eng ┆ cloud_pre ┆ skills_me ┆ job_url  │
│ ity       ┆ _level_ai ┆ degree_it ┆ certifica ┆   ┆ lish      ┆ ference   ┆ ntioned   ┆ ---      │
│ ---       ┆ ---       ┆ ---       ┆ tions     ┆   ┆ ---       ┆ ---       ┆ ---       ┆ str      │
│ str       ┆ str       ┆ str       ┆ ---       ┆   ┆ str       ┆ str       ┆ list[str] ┆          │
│           ┆           ┆           ┆ str       ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ High      ┆ Mid-Senio ┆ No        ┆ No        ┆ … ┆ No        ┆ GCP       ┆ ["Develop ┆ https:// │
│           ┆ r         ┆           ┆           ┆   ┆           ┆           ┆ pipelines ┆ www.link │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆

In [12]:
insert_into_db(df_genai, "genai_table", "my_project.duckdb", "genai_data")


--- Performing UPSERT for table 'genai_table' on key 'job_url' ---
Registered DataFrame as view 'df_upsert_view'
Executed: CREATE TABLE IF NOT EXISTS genai_table(
                                    job_url VARCHAR PRIMARY KEY,
                                    task_clarity VARCHAR,
                                    seniority_level_ai VARCHAR,
                                    requires_degree_it VARCHAR,
                                    mentions_certifications VARCHAR,
                                    years_of_experience VARCHAR,
                                    is_in_english VARCHAR,
                                    cloud_preference VARCHAR,
                                    skills_mentioned VARCHAR[]
                                    );
                                     (if table didn't exist)

Executing UPSERT SQL:

        INSERT INTO genai_table ("task_clarity", "seniority_level_ai", "requires_degree_it", "mentions_certifications", "years_of_experience", 

In [23]:
con.close()

In [22]:
con.sql("""
        SELECT * 
        FROM test_table

        JOIN genai_table
        ON test_table.job_url = genai_table.job_url
        """)

┌───────────────────────────────────────────┬─────────┬─────────────────────────────────────────────────┬────────────┬────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [17]:
con.sql("""
    SHOW TABLES        
""")

┌─────────────┐
│    name     │
│   varchar   │
├─────────────┤
│ genai_table │
│ job_tables  │
│ test_table  │
└─────────────┘